In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pysde import integrators, schemes, stochastic_integrals, storages

In [ ]:
# Settings for integration
start_time=0
step_size=1e-3
num_steps=501
initial_state=np.array([[1, 0],
                        [0, 1]])

# Settingss for stochastic integral integration
noise_dim = 2
seed = 0

# Settings for storage
chunk_size = 100
save_directory = 'test_dir/result_data'

# Drift function
def drift(current_state: np.ndarray, current_time: float):
    return -current_state

# Diffusion function
def diffusion(current_state: np.ndarray, current_time: float):
    return np.ones_like(current_state)

In [ ]:
# Stochastic integral generator
stochastic_integral = stochastic_integrals.DefaultStochasticIntegral(noise_dim, seed)

# Integration scheme, equipped with SDE model and stochastic integral generator
scheme = schemes.ExplicitEulerMaruyamaScheme(drift, diffusion, stochastic_integral)

# Storage model
storage = storages.NumpyStorage(save_directory)

# Outer loop integrator
integrator = integrators.StaticIntegrator(scheme, storage)

# Solve SDE
time_array, result_array = integrator.run(start_time, step_size, num_steps, initial_state)

In [ ]:
# Plot trajectories
num_components = result_array.shape[0]
num_trajectories = result_array.shape[1]

for i in range(num_components):
    _, ax = plt.subplots()
    ax.set_title(f"Component {i+1}")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(rf"$x_{{{i+1}}}(t)$")

    for j in range(num_trajectories):
        ax.plot(time_array, result_array[i, j, :])